## OneAtlas Data API methods including authentication, contract info, and search. Methods used here are described in the One Atlas Data API Reference: https://api.oneatlas.airbus.com/api-catalog/oneatlas-data/index.html

#### Get a bearer token

In [9]:
import requests
from json import loads, dumps
import getpass

api_key = getpass.getpass('Enter your API key: ')

url = "https://authenticate.foundation.api.oneatlas.airbus.com/auth/realms/IDP/protocol/openid-connect/token"
payload='client_id=IDP&grant_type=api_key&apikey=' + api_key
headers = {
  'Content-Type': 'application/x-www-form-urlencoded'
}
response = requests.request("POST", url, headers=headers, data=payload)
auth_header = 'Bearer ' + (loads(response.text)["access_token"])

Enter your API key: ········


#### Get contract information including the workspace ID

In [10]:
url = "https://data.api.oneatlas.airbus.com/api/v1/me"

payload={}
headers = {
  'Authorization': auth_header
}

my_info = requests.request("GET", url, headers=headers, data=payload)

contract_id = loads(my_info.text)["contract"]["id"]
workspace_id = loads(my_info.text)["contract"]["workspaceId"]

print(dumps(loads(my_info.text), indent=4, sort_keys=True))

{
    "_links": {
        "licenses": {
            "href": "https://data.api.oneatlas.airbus.com/api/v1/licenseAgreements"
        },
        "self": {
            "href": "https://data.api.oneatlas.airbus.com/api/v1/me"
        }
    },
    "contract": {
        "_links": {
            "cisContracts": {
                "href": "https://data.api.oneatlas.airbus.com/api/v1/contracts/9696b29d-2950-45f7-aa92-af96ec41e824/cisContracts"
            },
            "deliveries": {
                "href": "https://data.api.oneatlas.airbus.com/api/v1/contracts/9696b29d-2950-45f7-aa92-af96ec41e824/deliveries"
            },
            "orders": {
                "href": "https://data.api.oneatlas.airbus.com/api/v1/contracts/9696b29d-2950-45f7-aa92-af96ec41e824/orders"
            },
            "payments": {
                "href": "https://data.api.oneatlas.airbus.com/api/v1/contracts/9696b29d-2950-45f7-aa92-af96ec41e824/payments"
            },
            "reports": {
                "href"

#### Search the workspace

In [11]:
url = "https://search.foundation.api.oneatlas.airbus.com/api/v1/opensearch"

querystring = {"itemsPerPage":100, "startPage":1, "sortBy": "-publicationDate", "workspace": workspace_id}
headers = {
    'Cache-Control': 'no-cache',
    'Authorization': auth_header, 
    'Content-Type': 'application/json'}

response = requests.request('GET', url, headers=headers, params=querystring)

print(dumps(loads(response.text), indent=4, sort_keys=True))

{
    "error": false,
    "features": [
        {
            "_links": {
                "delete": {
                    "href": "https://search.foundation.api.oneatlas.airbus.com/api/v1/items/d6c4cafa-4577-4241-a130-ab40c927f9c7",
                    "name": "Delete",
                    "type": "HTTP"
                },
                "download": {
                    "href": "https://access.foundation.api.oneatlas.airbus.com/api/v1/items/d6c4cafa-4577-4241-a130-ab40c927f9c7/download",
                    "name": "Download",
                    "resourceId": "WO_000007286_1_SAL22005365-1_ACQ_PNEO4_00936001144780.zip",
                    "type": "HTTP"
                },
                "imagesGetBuffer": {
                    "href": "https://access.foundation.api.oneatlas.airbus.com/api/v1/items/d6c4cafa-4577-4241-a130-ab40c927f9c7/images/27409b97-5bc5-47f4-ac0f-b963cd27133a/buffer",
                    "name": "pms",
                    "resourceId": "27409b97-5bc5-47f4-ac0f-b96

#### Get select information about the workspace search results that are available for download

In [12]:
image_properties = []
image_geometries = []
image_hrefs = []
image_resource_ids =[]
image_names = []

i = 1
dl = True
rid = True
for feature in loads(response.text)['features']:
    image_properties.append(feature['properties'])
    image_geometries.append(feature['geometry'])
    try:
        if 'download' in feature['_links']:
            image_hrefs.append(feature['_links']['download']['href'])
            dl = True
    except:
        dl = False
    if dl == True:
        try:
            image_resource_ids.append(feature['_links']['download']['resourceId'])
            rid = True
        except:
            rid = False
    image_names.append(feature['properties']['sourceIdentifier'])
    i += 1
    dl = True
    rid = True
    
index = 0
for item in image_names:
    print(image_names[index], ' ', image_hrefs[index], ' ', image_resource_ids[index] )
    print(image_properties[index])
    print(image_geometries[index])
    print('\n')
index += 1

None   https://access.foundation.api.oneatlas.airbus.com/api/v1/items/d6c4cafa-4577-4241-a130-ab40c927f9c7/download   WO_000007286_1_SAL22005365-1_ACQ_PNEO4_00936001144780.zip
{'acquisitionDate': '2021-12-29T16:14:49.421Z', 'acquisitionIdentifier': None, 'activityId': 'b170c2fc-bf21-4ba9-95ba-21ecca19fe4c', 'azimuthAngle': 285.3573509915969, 'cloudCover': 0, 'commercialReference': 'WO_000007286_1', 'constellation': 'PNEO', 'contractId': 'CTR21000421', 'correlationId': 'PWOI_000007286_1_1_I_1', 'customerReference': 'EXT_Key_West_QC', 'expirationDate': '2023-01-24T14:13:43.343643057Z', 'format': 'SAFIR', 'id': 'd6c4cafa-4577-4241-a130-ab40c927f9c7', 'illuminationAzimuthAngle': 158.052182581519, 'illuminationElevationAngle': 38.92171338152981, 'incidenceAngle': 8.090429481765499, 'incidenceAngleAcrossTrack': -7.881424389452759, 'incidenceAngleAlongTrack': -2.010582972258341, 'internalLine': '1', 'internalReference': 'SAL22005365', 'lastUpdateDate': '2022-01-24T14:13:43.343638297Z', 'platf

#### Search the One Atlas Data catalog by acquisition date range, geometry, cloud cover, incident angle, etc.

In [5]:
url = "https://search.foundation.api.oneatlas.airbus.com/api/v1/opensearch"

payload="{\"itemsPerPage\":100,\"startPage\":1,\"productType\":\"bundle,mono\",\"acquisitionDate\":\"[2021-03-01T00:00:00.000Z,2021-04-01T23:59:59.999Z]\",\"cloudCover\":\"[0,10]\",\"incidenceAngle\":\"[0,33]\",\"sortBy\":\"-publicationDate\",\"workspace\":\"public\",\"relation\":\"intersects\",\"geometry\":{\"type\":\"Polygon\",\"coordinates\":[[[-9.176808297561358,38.728242233199325],[-9.16970342383137,38.728242233199325],[-9.16970342383137,38.73326393859847],[-9.176808297561358,38.73326393859847],[-9.176808297561358,38.728242233199325]]]}}"
headers = {
  'Authorization': auth_header,
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

print(dumps(loads(response.text), indent=4, sort_keys=True))

{
    "error": false,
    "features": [
        {
            "_links": {
                "delete": {
                    "href": "https://search.foundation.api.oneatlas.airbus.com/api/v1/items/1f2a70b8-e986-4482-8429-db238c034490",
                    "name": "Delete",
                    "type": "HTTP"
                },
                "imagesGetBuffer": [
                    {
                        "href": "https://access.foundation.api.oneatlas.airbus.com/api/v1/items/1f2a70b8-e986-4482-8429-db238c034490/images/931b7808-4ff7-49c9-a231-33eaaa6b1ffb/buffer",
                        "name": "multispectral",
                        "resourceId": "931b7808-4ff7-49c9-a231-33eaaa6b1ffb",
                        "type": "getBuffer"
                    },
                    {
                        "href": "https://access.foundation.api.oneatlas.airbus.com/api/v1/items/1f2a70b8-e986-4482-8429-db238c034490/images/0565a1cc-bb49-4270-a0f3-8694559a2892/buffer",
                        "nam

#### Get additional details about the subscription, contracts, orders, deliveries, reports, etc.

In [ ]:
# Use one of these endpoints as the URL for specific details

# 'https://data.api.oneatlas.airbus.com/api/v1/contracts/{}/cisContracts'.format(contract_id)
# 'https://data.api.oneatlas.airbus.com/api/v1/contracts/{}/deliveries'.format(contract_id)
# 'https://data.api.oneatlas.airbus.com/api/v1/contracts/{}/orders'.format(contract_id)
# 'https://data.api.oneatlas.airbus.com/api/v1/contracts/{}/payments'.format(contract_id)
# 'https://data.api.oneatlas.airbus.com/api/v1/contracts/{}/reports'.format(contract_id)
# 'https://data.api.oneatlas.airbus.com/api/v1/contracts/{}'.format(contract_id)
# 'https://data.api.oneatlas.airbus.com/api/v1/contracts/{}/subscriptions'.format(contract_id)

url = 'https://data.api.oneatlas.airbus.com/api/v1/contracts/{}/subscriptions'.format(contract_id)

headers = {
  'Authorization': auth_header
}

response = requests.request("GET", url, headers=headers)
print(dumps(loads(response.text), indent=4, sort_keys=True))